In [2]:
%matplotlib inline
import sklearn
import nose
import astroML_addons
import astroML
import scipy
import pandas
import xdgmm
from astropy.io import fits
import csv
import os
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt



#xdgmm.read_model('galaxyTrialSet'+datestr+'.fit')
xdgmmG = xdgmm.XDGMM(filename='galaxyTrialSet2018-03-22.fit')

#xdgmm.save_model('notNullTrialset_feb15.fit')
xdgmmQ = xdgmm.XDGMM(filename='notNullTrialset_feb15.fit')

print("Galaxy weights:")
print(xdgmmG.weights)
print("Quasar weights:")
print(xdgmmQ.weights)

Galaxy weights:
[  1.30442655e-01   1.31654676e-01   2.15592659e-01   5.49861817e-03
   2.13396564e-01   2.77492505e-02   3.23011640e-02   2.43127793e-01
   2.36619802e-04]
Quasar weights:
[ 0.35075201  0.16394932  0.32481524  0.0415381   0.11894532]


In [15]:
def fetchEpochVals(epoch, directorystr):
    
    deltaW = []
    sigW = []
    mjd = [] #readdata 0
    w1 = [] #readdata 1
    w2 = [] #readdata 2
    w1s = [] #readdata 3
    w2s = [] #readdata 4
    name = []
    

    #directorystr = './csvfiles/notNull_feb15/'
    directory = os.fsencode(directorystr)
    filelist = os.listdir(directory)
    #print(len(filelist))
    readdata = np.zeros((6,len(filelist))) #one big list to rule them all
    fileCount = 0 #to keep our place in each list as we go through the files
    errCount = 0
    for file in filelist:
        
        filestr = os.fsdecode(file)
        if(filestr.endswith(".csv")):
            readfile = open(directorystr+filestr, 'r')
            reader = csv.reader(readfile)
            typeIndex = 0 #to iterate through the w1, w2, etc list
            nullVals = 0 #check if the column has nulls
            
            try:
                for row in enumerate(reader):
                    
                    if(row[1][epoch] != 'null'):
                        readdata[typeIndex][fileCount] = row[1][epoch]        
                    else:
                        nullVals = nullVals + 1
                        #print("Caught a null in " + filestr)
                    typeIndex = typeIndex + 1
                filenamestr = filestr.split(".c") #splits into <radec> and sv
                #name.append(filenamestr[0]) ###
                #name.insert(fileCount, filenamestr)
                if(nullVals == 0):
                    name.append(filenamestr[0])
                    fileCount = fileCount + 1 #only increment if whole col not null   
            except IndexError:
                #print("IndexError")
                #print(filestr)
                #print(row[1])
                errCount = errCount + 1
                continue
    print("Error count : " + str(errCount))
                    
    w1 = [float(x) for x in readdata[1]]
    w2 = [float(x) for x in readdata[3]]
    w1s = [float(x) for x in readdata[2]]
    w2s = [float(x) for x in readdata[4]]
    mjd = [float(x) for x in readdata[0]]
    #print(readdata.shape)
    
    
    deltaW = [x - y for x,y in zip(w1,w2)]
    sigW = [np.sqrt(sx**2 + sy**2) for sx,sy in zip(w1s, w2s)]
    
    return deltaW, sigW, fileCount, name, mjd

def checkProbs(epoch, modelone, modeltwo, directorystr):

    result = fetchEpochVals(epoch, directorystr)
    deltaW = result[0]
    sigW = result[1]
    npoints = result[2]
    radec = result[3]
    mjd = result[4]

    deltaW = np.array(deltaW)
    deltaW = deltaW.reshape(deltaW.shape[0], 1)

    sigW = np.array(sigW)
    sigW = sigW.reshape(sigW.shape[0], 1,1)

    tryProb = modelone.score_samples(deltaW, sigW)

    maxindex = np.argmax(tryProb[0])

    dubious = [x for x in tryProb[0] if x>0]

    minindex = np.argmin(tryProb[0])

    lowprob = [x for x in tryProb[0] if x < -3] #the low values
    
#     for x in tryProb[0]:
#         probModelTwo = modeltwo.score_samples()

    lowprobRADEC = []
    lowDeltaWs = []
    lowSigWs = []
    lowMjds = []
    for lowval in lowprob:
        for val in tryProb[0]:
            if(val == lowval):
                minindex = np.where(tryProb[0] == val) #indices of the low values
                for i in minindex:
                    if(len(deltaW[i])!=0):
                        print(deltaW[i][0])
                        print(sigW[i][0])
                        print(mjd[i])
                        lowprobRADEC.append(radec[i])
                        lowDeltaWs.append(deltaW[i][0])
                        lowSigWs.append(sigW[i][0])
                        lowMjds.append(mjd[i])
                    else:
                        continue
                    
    if(len(lowDeltaWs) != 0):
        lowDeltaWs = np.array(lowDeltaWs)
        lowSigWs = np.array(lowSigWs)
        return lowprobRADEC,
        
#         lowDeltaWs.reshape(lowDeltaWs.shape[0], 1)
#         lowSigWs.reshape(lowSigWs.shape[0], 1, 1)
#         probModeltwo = modeltwo.score_samples(lowDeltaWs, lowSigWs)               
#         return lowprobRADEC, probModeltwo[0]
    else:
        return

print("running")
directorystr = './csvfiles/notNull_feb15/'
model = xdgmmQ #galaxies or quasars?
model2 = xdgmmG




lowProbVals = []
crossmodelprobs = []
names = []
import subprocess

for epoch in range(1, 10):
    print("Epoch: " + str(epoch))
    #valsOut = checkProbs(epoch, xdgmmQ, directorystr)
    result = checkProbs(epoch, model, model2, directorystr)
    if(result):
        print(result)
        for i in result[0]:
            names.append(i)
        
        
#         for i in result[1]:
#             if i > -2:
#                 crossmodelprobs.append(i) #a list of the most cross-model-esque candidates
# plt.plot(crossmodelprobs)
    
#     for name in valsOut: ###for visual inspection
#         namestring = directorystr + name + ".csv"
#         #subprocess.call(['open', '-a', 'TextEdit', namestring])
#         readfile = open(namestring, 'r')
#         reader = csv.reader(readfile)
#         #for row in enumerate(reader):
#             #mjd = row[epoch]
#             #print(row[1])
#             #print(mjd)
print(len(names))            
print(names)

print("done.")
#not all files have the same number of observations


running
Epoch: 1
Error count : 0
[ 5.589]
[[ 0.18559364]]
55369.8752782
[ 4.681]
[[ 0.13572398]]
55370.2720622
(['2.01734154825_-0.694226157473', '2.20724795619_-0.278045983738'],)
Epoch: 2


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:105: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:106: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:109: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


Error count : 0
[ 2.837]
[[ 0.10385567]]
55370.205974
(['2.21690200262_-0.461093741689'],)
Epoch: 3
Error count : 0
[ 3.924]
[[ 0.20280533]]
55389.5164577
(['7.65298424529_28.7194374038'],)
Epoch: 4
Error count : 0
[ 2.642]
[[ 0.1716042]]
55375.4291073
[-0.74]
[[ 0.25751311]]
55379.3984696
[ 3.692]
[[ 0.19811613]]
55389.6487614
(['2.2708671093_14.2080316544', '6.33949785821_13.5136896585', '7.65298424529_28.7194374038'],)
Epoch: 5
Error count : 0
[ 2.187]
[[ 0.08335466]]
55367.4945743
(['1.43630791759_-4.82091624004'],)
Epoch: 6
Error count : 0
Epoch: 7
Error count : 0
[ 4.373]
[[ 0.15273834]]
55370.2058466
[ 5.037]
[[ 0.0825409]]
55370.0736702
(['2.20724795619_-0.278045983738', '2.21690200262_-0.461093741689'],)
Epoch: 8
Error count : 0
[ 4.854]
[[ 0.06958448]]
55370.1397585
(['2.21690200262_-0.461093741689'],)
Epoch: 9
Error count : 0
[ 3.775]
[[ 0.28977405]]
55369.9413664
[ 3.827]
[[ 0.19278226]]
55370.7351896
[ 2.894]
[[ 0.24496939]]
55554.7668262
[ 3.66]
[[ 0.37915828]]
55388.6565

In [17]:
f = open('quasar_flags_-5.txt', 'w')
for name in names:
    f.write(name+'\n')
f.close()
print("Done")

Done


In [5]:
def compareOddValuesToModel(oddvalues, model):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-230d83155282>, line 2)

In [ ]:
def fetchEpochValsG(directorystr):
    
    deltaW = []
    sigW = []
    w1 = []
    w2 = []
    w1s = []
    w2s = []
    mjd = []
    name = []
    

    #directorystr = './csvfiles/galaxy_csvfiles-150318/'
    directory = os.fsencode(directorystr)
    filelist = os.listdir(directory)
    print(len(filelist))
    readdata = np.zeros((22,len(filelist))) #one big list to rule them all
    fileCount = 0 #to keep our place in each list as we go through the files
    blankCount = 0
    for file in filelist:
        filestr = os.fsdecode(file)
        if(filestr.endswith(".csv")):
            readfile = open(directorystr+filestr, 'r')
            #name.append(filestr)
            reader = csv.reader(readfile)
            typeIndex = 0 #to iterate through the w1, w2, etc list
            nullCount = 0
            rowCount = 0
            try:
                for row in enumerate(reader):
                    entry = row[1][0]
                    try:
                        fentry = float(entry)
                    except ValueError:
                        if entry != 'null':
                            #print("String!")
                            rowCount = rowCount + 1 #keep track of where we are in the file
                            continue #skip to next row, do not increment readdata row
                        else:
                            #print("Null!")
                            nullCount = nullCount + 1 #this is a null in W1 or W2. don't want this file
                    
                    #print(fentry)
                    readdata[typeIndex][fileCount] = fentry
                    rowCount = rowCount + 1 #move on to the next row in the file
                    typeIndex = typeIndex + 1 #move to next row in the data array
                    if rowCount == 14:
                        break #move on to next file.
                if nullCount == 0:
                    fileCount = fileCount + 1 #overwrite if we found null in W1, W2.
                    name.append(filestr)
            except IndexError:
                #print(readdata[0][fileCount])
                #print(filestr)
                #print("IndexError")
                blankCount = blankCount + 1
                continue
                #break
                
    print(blankCount)       
    w1 = [float(x) for x in readdata[1]]
    #print(w1)
    w2 = [float(x) for x in readdata[3]]
    #print(w2)
    w1s = [float(x) for x in readdata[2]]
    #print(w1s)
    w2s = [float(x) for x in readdata[4]]
    #print(w2s)
    
    #print(name)
    
    deltaW = [x - y for x,y in zip(w1,w2)]
    sigW = [np.sqrt(sx**2 + sy**2) for sx,sy in zip(w1s, w2s)]
    
    notNullCount = 0
    for val in w1:
        if val != 0:
            notNullCount = notNullCount + 1
    deltaW = deltaW[:notNullCount]
    sigW = sigW[:notNullCount]
    
    return deltaW, sigW, notNullCount, name

def findUnlikelyValues(epoch, directorystr):
    result = fetchEpochVals(epoch, directorystr)
    deltaW = result[0]
    sigW = result[1]
    npoints = result[2]
    radec = result[3]

    deltaW = np.array(deltaW)
    deltaW = deltaW.reshape(deltaW.shape[0], 1)

    sigW = np.array(sigW)
    sigW = sigW.reshape(sigW.shape[0], 1,1)

    tryProb = xdgmm.score_samples(deltaW, sigW)

    maxindex = np.argmax(tryProb[0])

    dubious = [x for x in tryProb[0] if x>0]

    minindex = np.argmin(tryProb[0])

    lowprob = [x for x in tryProb[0] if x < -5]

    lowprobRADEC = []
    for lowval in lowprob:
        for val in tryProb[0]:
            if(val == lowval):
                minindex = np.where(tryProb[0] == val)
                for i in minindex:
                    lowprobRADEC.append(radec[i])
                    
    return lowprobRADEC